<a href="https://colab.research.google.com/github/NathaliaOliveir/CEE2/blob/main/Trabalho1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho 1: Análise da Taxa SELIC com Python

**Curso**: Estatística UnB    
**Disciplina**: Computação Estatística 2 em Python    
**Aluna**: Nathalia Oliveira Moreira    
**Matrícula**: 200042289    
**Professora**: José Augusto Fiorucci     
**Data de Entrega**: 22 de junho de 2025

---

## Tema do Trabalho

Este relatório tem como objetivo utilizar a linguagem Python para acessar dados reais da Taxa SELIC através da API do Banco Central do Brasil, realizando análises e visualizações que ajudem a compreender o comportamento da economia brasileira ao longo do tempo.

---

## Introdução

A **Taxa SELIC** (Sistema Especial de Liquidação e Custódia) é a taxa básica de juros da economia brasileira, utilizada como principal instrumento de política monetária pelo Banco Central do Brasil. Essa taxa influencia diretamente as demais taxas de juros praticadas no mercado, como as cobradas em empréstimos, financiamentos e aplicações financeiras.

Em termos práticos, quando o Banco Central aumenta a SELIC, o objetivo principal é conter a inflação; por outro lado, quando a taxa é reduzida, busca-se estimular a economia, incentivando o consumo e o investimento.

Monitorar a evolução da SELIC ao longo do tempo é fundamental para entender o comportamento da economia brasileira, seus ciclos de crescimento e retração, e os impactos sobre a vida da população.

Neste relatório, utilizaremos a linguagem Python para acessar dados reais da Taxa SELIC disponibilizados pela API do Banco Central, realizar análises estatísticas e gerar visualizações gráficas.
